In [1]:
import pandas as pd
import cv2 as cv
import math
import numpy as np 
import adjmtrx_io


In [3]:
img = cv.imread("./images/testimg.jpeg")

In [4]:
gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 

In [5]:

def sim_exp(a,i1,i2):
    return math.exp((-a)*(i1-i2)^2)

def get_1d_ind(r,c,width):
    return r*width + c



def get_adjacency_image(raw_img):
    rows,cols = raw_img.shape
    adj_mtrx = np.zeros((rows*cols,rows*cols))
    s = rows*cols
    computed = set()
    for curr_r in range(rows):
        for curr_c in range(cols):
            pix = get_1d_ind(curr_r,curr_c,cols)
            if not pix in computed:
                ##go to right as far as possible
                nrp = pix +1 #"neighbouring right pixel"
                offset = 1
                while (nrp < s) and (curr_c+offset < cols) and abs(nrp-pix) < 9: 
                    sim = sim_exp(30,raw_img[curr_r,curr_c],raw_img[curr_r,curr_c+offset])
                    adj_mtrx[nrp,pix] = sim #similarity between nrp and current pixel
                    adj_mtrx[pix,nrp] = sim #weighting from pix to  nrp is symmetric
                    computed.add(nrp)
                    nrp+=1
                    offset+=1

                ##go to left as far as possible
                nlp = pix-1 #"neighbouring left pixel"
                offset = 1
                while (nlp >= 0) and (curr_c-offset > 0) and (abs(nlp-pix) < 9):
                    sim = sim_exp(30,raw_img[curr_r,curr_c],raw_img[curr_r,curr_c-offset])
                    adj_mtrx[nlp,pix] = sim #similarity between nrp and current pixel
                    adj_mtrx[pix,nlp] = sim #weighting from pix to  nrp is symmetric
                    computed.add(nlp)
                    nlp-=1
                    offset+=1

                ##go up as far as possible
                nup = get_1d_ind(curr_r-1,curr_c,cols) #"neighbouring upper pixel"
                offset = 1
                while (nup >= 0) and (abs(nup-pix) < 9):
                    sim = sim_exp(30,raw_img[curr_r,curr_c],raw_img[curr_r-offset,curr_c])
                    adj_mtrx[nup,pix] = sim #similarity between nrp and current pixel
                    adj_mtrx[pix,nup] = sim #weighting from pix to  nrp is symmetric
                    computed.add(nup)
                    offset+=1
                    nup = get_1d_ind(curr_r-offset,curr_c,width)

                ##go down as far as possible 
                nbp = get_1d_ind(curr_r+1,curr_c,cols) #"neighbouring belwo pixel"
                offset = 1
                while (nbp < s) and (abs(nbp-pix) < 9):
                    sim = sim_exp(30,raw_img[curr_r,curr_c],raw_img[curr_r+offset,curr_c])
                    adj_mtrx[nbp,pix] = sim #similarity between nrp and current pixel
                    adj_mtrx[pix,nbp] = sim #weighting from pix to  nrp is symmetric
                    computed.add(nbp)
                    offset+=1
                    nbp = get_1d_ind(curr_r+offset,curr_c,cols)

                computed.add(pix)
    return adj_mtrx
A = get_adjacency_image(gray_img)


/var/folders/4j/qzg2kvtj7qg6pp3nbyhr772c0000gn/T/ipykernel_1248/4153791463.py:2: RuntimeWarning: overflow encountered in scalar subtract
  return math.exp((-a)*(i1-i2)^2)
